<a href="https://colab.research.google.com/github/melrahmtz/purple-box/blob/main/hands-on-practice/pdfparsing_docling_ollama_glider_justtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://medium.com/the-ai-forum/building-an-advanced-rag-pipeline-using-docling-groq-ollama-with-glider-evaluation-b91c290b0c84

In [1]:
!pip install docling docling-core --quiet
!pip install langchain --quiet
!pip install langchain-text-splitters --quiet
!pip install langchain-huggingface --quiet
!pip install langchain-chroma --quiet
!pip install langchain-ollama --quiet
!pip install langchain-groq --quiet
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 k

# Setup LLM

In [2]:
import  os
from getpass import getpass
os.environ['GROQ_API_KEY'] = getpass('GROQ_API_KEY')
os.environ['HUGGINGFACE_API_KEY'] = getpass('HF_TOKEN')

## Ollama LLM
from langchain_ollama.llms import OllamaLLM
ollama_llm = OllamaLLM(model='mistral:7b')

## Groq LLM
from langchain_groq import ChatGroq
groq_llm = ChatGroq(model="mixtral-8x7b-32768",
                    temperature=0,
                    max_tokens=None,
                    timeout=None,
                    max_retries=5,)

GROQ_API_KEY··········
HF_TOKEN··········


# Setup Embedding Model

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

HF_EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=HF_EMBED_MODEL_ID)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Document Parsing and Formatting

In [4]:
# Setup Custom Document Loader using Docling and extract data in Markdown Format

from typing import Iterator, List, Tuple
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument

from docling.document_converter import DocumentConverter

class DoclingPDFLoader(BaseLoader):
    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)

In [5]:
#FILE_PATH =  "https://arxiv.org/pdf/2312.10997"
FILE_PATH = "Manuale ROTOMARR.pdf"

In [6]:
# Download Data into Langchain Document Schema
from IPython.display import display, Markdown

loader = DoclingPDFLoader(file_path=FILE_PATH)
docs = loader.load()

display(Markdown(docs[0].page_content))

<!-- image -->

Revisione 00.00 - Febbraio 2015

GIRACASTAGNE AUTOMATICO ROTOMARR

<!-- image -->

## 1. INFORMAZIONI SULLA SICUREZZA

Il  presente  manuale  contiene  indicazioni  ed  informazioni fondamentali per il corretto utilizzo del GIRACASTAGNE AUTOMATICO (CUOCI CALDARROSTE) ROTOMARR .

- -Leggere il manuale  nella sua completezza per comprendere l'utilizzo della macchina;
- -Tenere questo manuale per future consultazioni in un luogo sicuro;
- -Osservare le istruzioni indicate in questo manuale per garantire la sicurezza dell'utilizzatore;
- -La non osservanza delle indicazioni elencate in questo manuale comporterà l'annullamento della garanzia;
- -MECTRONICA S.r.l. non è responsabile per danni o lesioni causate dalla non osservanza delle informazioni elencate nel presente manuale.

## 2. SPECIFICHE

Il  GIRACASTAGNE  AUTOMATICO ROTOMARR entra in funzionamento grazie ad un alimentatore elettrico con le seguenti caratteristiche:

Voltaggio di alimentazione:

12 Volts DC

Amperaggio:

1 A

Dimensione connessione:

5,5*2,1mm DC Plug

## 3. DESTINAZIONE D'USO E UTILIZZATORI

<!-- image -->

La  macchina  in  oggetto  è  destinata  ad  un pubblico  non  professionale,  ma  per  un utilizzatore domestico e quindi le attività di montaggio, assemblaggio, manutenzione, riparazione sono vietate in quanto l'utilizzatore finale non ha le capacità tecniche per portarle a termine. Sono  previste  e  ammesse  da  parte  dell'utilizzatore finale  solo  le  attività  di  regolazione  descritte  al

successivo punto 4 .

La  macchina  è  stata  progettata  per  la  cottura  su  fiamma delle  castagne  al  fine  di  produrre  'caldarroste'  e  va collocata  su  di  un  fornello  di  tipo  domestico  (diametro massimo:  70  cm)  .  La  macchina  infatti  dispone  di  un motorino elettrico che mette in rotazione il cestello entro il  quale sono inerite le castagne e ne garantisce la cottura uniforme.

- È  tassativamente  proibito  utilizzare  nel  cestello  altro materiale che non siano castagne (materiali infiammabili o combustibili,  altri  alimenti,  materiali  tessili  o  plastici,  o comunque    materiali  che  possono  produrre  diossine mediante riscaldamento).

È tassativamente proibito utilizzare la macchina in modo difforme  da  quanto  riportato  nel  manuale  di  uso  e manutenzione

Non sono consentiti:

- 1) l'utilizzo della  macchina  con  altro  materiale rispetto alle castagne,

<!-- image -->

## ROTOMARR

- 2) l'utilizzo della macchina su fornelli di dimensioni superiori ai 70 cm di diametro
- 3) il lavaggio della macchina in lavastoviglie
- 4) il lavaggio  della  macchina  con  getto  d'acqua pieno
- 5) l'apertura dei ripari o una qualsiasi manomissione della macchina
- 6) l'utilizzo all'esterno in caso di cattive condizioni meteorologiche  (pioggia,  neve,  grandine,  vento forte)
- 7) l'utilizzo  in  locali  con  pericolo  di  esplosione  o incendio  o  in  presenza  di  grandi  quantitativi  di materiale infiammabile

## 3. REGOLAZIONE

<!-- image -->

Non  effettuare  nessuna  regolazione  senza prima aver letto e compreso tutte le istruzioni presenti su questo manuale.

- 1. Assicurarsi  che  l'alimentatore  di  corrente  non  sia collegato all'apparecchio o alla corrente;
- 2. Svitare  il  pomello  ( A )  in  senso  antiorario  in  maniera tale che il supporto ( B ) sia completamente lento.
- 3. Regolare l'altezza  del  cestello  ( C )  a  circa  4-  5  cm  dal piano di cottura;
- 4. Ruotare il  pomello  ( A )  in  senso  orario  fin  quando  la staffa di supporto ( D )  non è completamente fissata e stabile;
- 5. Qualora  la  staffa  risultasse  instabile,  rimuovere  la griglia dal piano di cottura.

<!-- image -->

## 4. AVVIAMENTO

<!-- image -->

Prima  di  caricare  il  prodotto,  eseguire  una profonda incisione sulle castagne\caldarroste

per evitare scoppi che possono provocare danni.

<!-- image -->

Evitare  possibili  urti  all'apparecchio  che possano compromettere la corretta rotazione

o il normale funzionamento.

<!-- image -->

<!-- image -->

Utilizzare  l'apparecchio  solo  su  un  fornello  a gas dal diametro max. di 70mm.

Non regolare l'altezza del cestello durante la cottura.

- 1. Riempire  la  bocca  del  cestello  con  le  castagne  già incise  facendo  attenzione  che  siano  uniformemente distribuite nel cestello
- 2. Collocare  l'apparecchio  sulla  griglia  del  piano  di cottura;

<!-- image -->

- 3. Collegare l'alimentatore di corrente nell'apposita presa ( 1 ) dell'apparecchio e poi collegare la spina dell'alimentatore  alla  presa  di  corrente.  In  questa operazione fare molta attenzione ad evitare che il cavo stazioni sopra o nelle vicinanze della fiamma al fine di evitare bruciature o surriscaldamenti dello stesso;
- 4. Verificare  che  l'impianto  elettrico  cui  la  macchina  è collegata sia dotato di interruttore magnetotermico di adeguata corrente nominale e di interruttore differenziale con I ∆ n non superiore a 0,03 A
- 5. Attivare  l'apparecchio  premendo  il  tasto  verde  ( 2 ), accertandosi che non ci sia nulla che possa compromettere  la  corretta  rotazione  o  che  oggetti esterni possano rimanere incastrati durante la normale rotazione;
- 6. Accendere il fornello a fiamma massima; assicurarsi che il fondo del cestello sia ben centrato con la fiamma;

<!-- image -->

Non superare i 1200 gr. di prodotto all'interno del cestello.

- cottura per altri 4 o 5 minuti. 7. Lasciare in cottura per circa 20 minuti. Qualora il prodotto non risulti ancora pronto, continuare la
- 8. Spegnere l'apparecchio con il tasto nero ( 3 ) e attendere qualche secondo a fornello spento prima di rimuovere gli alimenti dal cestello;

<!-- image -->

- 9. Rimuovere il prodotto afferrando la barra orizzontale, che funge da maniglia ( 4 ).

## 5. PULIZIA & MANUTENZIONE

Al  termine  di  ogni  utilizzo,  staccare  la  spina  dal  muro  e rimuovere l'alimentatore elettrico ( 1 ).

Prima di re-inscatolare l'apparecchio, attendere il raffreddamento del cestello.

A raffreddamento  avvenuto,  rimuovere  l'apparecchio  dal piano cottura e strofinare con un panno umido il cestello, rimuovendo briciole o detriti rimasti al suo interno.

Lavare solo la parte del cestello.

Non  lavare  con  getto  d'acqua  piena  e  non  mettere  in lavastoviglie.

L'apparecchio non necessita alcun tipo di manutenzione. Non  rimuovere  il  carter  di  chiusura  che  protegge  le componenti elettriche.

## 6. RICAMBI

Nelle  seguenti  pagine  saranno  indicate  a  disegno  le componenti meccaniche con i loro codici.

Qualora  sia  necessario  ordinare  una  componente  per richiedere  un  ricambio,  è  necessario  indicare  quantità  e codice di riferimento.

## 7. CONTATTI

<!-- image -->

Via Monari Sardè, 8

40010 Bentivoglio (BO) Italia

Tel. +39 0516641440 Fax. +39 0518909108

E-mail. info@mectronica.it Web: www.mectronica.it

P. IVA e C.F. 031228011206

## 8. GARANZIA

Le specifiche sono soggette a  modifiche senza  preavviso da parte del fabbricante.

© Copyright 2015 MECTRONICA S.r.l.

MECTRONICA  garantisce  la  conformità  dei  prodotti forniti.  La  garanzia  per  vizi  è  limitata  ai  soli  difetti  dei prodotti conseguenti a difetti di progettazione, di materiale o di costruzione riconducibili a MECTRONICA. La garanzia non comprende:

- -  Avarie  o  danni  causati  dal  trasporto  o  da  anomalie dell'impianto elettrico ove viene utilizzato l'apparecchio, o difettosa installazione e/o qualsiasi utilizzo non adeguato e non espressivamente indicato nel manuale d'uso.
- - Manomissione o danni causati dall'impiego di pezzi e/o ricambi non originali.
- -Difetti  e/o  danni  causati  da  agenti  chimici  e/o atmosferici (es. umidità da esterno, ghiaccio, prodotti per pulizia corrosivi/chimici, ecc.).

La garanzia ha una durata di 24 mesi decorrenti dalla data della  vendita.  La  garanzia  è  subordinata  all'espressa richiesta  scritta  a  MECTRONICA  tramite  raccomandata con ricevuta di ritorno.

| 3ROTOMARR   | 3ROTOMARR          | GIRACASTAGNE AUTOMATICO ROTOMARR      | 1    |
|-------------|--------------------|---------------------------------------|------|
| Posizione   | Codice             | Descrizione                           | Q.tà |
| 1           | 2ROTOMARR3         | BASE SUPPORTO ROTOMARR                | 1    |
| 2           | 2ROTOMARR5         | SUPPORTO ROTOMARR                     | 1    |
| 3           | 28018MAR0111B      | CARTER CHIUSURA                       | 1    |
| 4           | 28018MAR0110       | PIASTRA MOTORIDUTTORE                 | 1    |
| 5           | 2ROTOMARR1         | CESTELLO ROTOMARR                     | 1    |
| 6           | 2MAR0111E          | ALBERO DI TRASMISSIONE                | 1    |
| 7           | 2MAR0111D          | PROTEZIONE                            | 1    |
| 8           | 2MAR0111C          | PIASTRA PLEXIGLASS 60/10              | 1    |
| 9           | 106F1Z1000000010   | DADO ESAGONO B FEZIN M10 UNI 5589     | 2    |
| 10          | 138H000006000002RS | CUSCINETTO                            | 2    |
| 11          | 1680000BS138F12208 | MOTORIDUTTORE                         | 1    |
| 12          | 131000320106030    | MANOPOLA                              | 1    |
| 13          | 104FEZ10100040006  | GRANO PUNTA PIANA ZIN M4x6 UNI 5923   | 1    |
| 14          | 114F1Z7010029006   | VITE AUT ZIN TCICr @ 2.9x6            | 3    |
| 15          | 100F1Z1020003005   | VITE TCICr FE ZIN M3x5 UNI 7985       | 2    |
| 16          | 101F1Z1000003015   | VITE TCICr FE ZIN M3x015 UNI 5739     | 1    |
| 17          | 106F1Z1010000003   | DADO ESAGONO FEZIN M3 UNI 5588        | 1    |
| 18          | 199A0CE00042060    | TARGHETTA CE ROTOMARR                 | 2    |
| 19          | 129A000000000310   | RIVETTO AL 03x010                     | 1    |
| 20          | 24040849679513     | ALIMENTATORE 12V DC PLUG 5.5*2.1mm 1A | 1    |

<!-- image -->

<!-- image -->

# Split the document into smaller Chunks

In [7]:
from langchain_text_splitters import MarkdownTextSplitter

mD_splitter = MarkdownTextSplitter(chunk_overlap=200)
doc_text = docs[0].page_content
md_split = mD_splitter.split_text(doc_text)

for items in md_split:
    print(f"Total Number of Characters :{len(items)}")
    print(f"Total Number of Words :{len(items.split(' '))}")
    print(f"Total Number of Tokens :{len(items.split()) *(4/3)}")
    print("------------------------------------------------")

Total Number of Characters :3986
Total Number of Words :700
Total Number of Tokens :765.3333333333333
------------------------------------------------
Total Number of Characters :3934
Total Number of Words :670
Total Number of Tokens :768.0
------------------------------------------------
Total Number of Characters :2165
Total Number of Words :965
Total Number of Tokens :384.0
------------------------------------------------


In [8]:
from IPython.display import display, Markdown
print(md_split[0])

<!-- image -->

Revisione 00.00 - Febbraio 2015

GIRACASTAGNE AUTOMATICO ROTOMARR

<!-- image -->

## 1. INFORMAZIONI SULLA SICUREZZA

Il  presente  manuale  contiene  indicazioni  ed  informazioni fondamentali per il corretto utilizzo del GIRACASTAGNE AUTOMATICO (CUOCI CALDARROSTE) ROTOMARR .

- -Leggere il manuale  nella sua completezza per comprendere l'utilizzo della macchina;
- -Tenere questo manuale per future consultazioni in un luogo sicuro;
- -Osservare le istruzioni indicate in questo manuale per garantire la sicurezza dell'utilizzatore;
- -La non osservanza delle indicazioni elencate in questo manuale comporterà l'annullamento della garanzia;
- -MECTRONICA S.r.l. non è responsabile per danni o lesioni causate dalla non osservanza delle informazioni elencate nel presente manuale.

## 2. SPECIFICHE

Il  GIRACASTAGNE  AUTOMATICO ROTOMARR entra in funzionamento grazie ad un alimentatore elettrico con le seguenti caratteristiche:

Voltaggio di alimentazione:

12 Volts DC

Amperag

# Build Index and Load Embedding

In [9]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_texts(texts=md_split,
                                embedding=embeddings,
                                collection_name="rag",
                                collection_metadata={"hnsw:space":"cosine"},
                                persist_directory="chromadb")

# Check if the documents  have been loaded
print(len(vectorstore.get()['documents']))

3


# Context-Aware Retrieval

In [10]:
# The information most relevant to a query may be buried in a document with a lot of irrelevant text.
# Passing that full document through our application can lead to more expensive LLM calls and poor response.
# The idea of Contextual Compression is to compress the retrieved relevant document so that only relevant information is returned.

from langchain_chroma import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

# Load the vectorstore from the persisted directory
vectorstore = Chroma(persist_directory="chromadb", embedding_function=embeddings)

retriver = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":2})
compressor = LLMChainFilter.from_llm(llm=ollama_llm)

compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriver)

# Retriever RAG Pipeline and Compressor Retriever RAG Chain

In [11]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  13013      0 --:--:--  0:00:01 --:--:-- 13021
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [12]:
import subprocess
import time
import threading

def start_ollama():
    """Starts the Ollama server in the background."""
    process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Wait a few seconds for Ollama to start
    time.sleep(5)
    return process

# Start Ollama in a separate thread
ollama_process = start_ollama()

# ... rest of your code ...

In [13]:
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff8

In [14]:
from typing import Iterable
from langchain_core.documents import Document as LCDocument
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs: Iterable[LCDocument]):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    "You are a customer support chatbot. Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {question}\nAnswer:\n"
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)


In [15]:
print(rag_chain.invoke("Quali materiali è proibito utilizzare nel cestello della macchina?"))  # What materials are prohibited from being used in the machine's basket?


 Nel cestello di una macchina, sono proibiti l'utilizzo di materiali infiammabili o materiali che possono causare danni alla macchina. Non è consigliabile utilizzare materiali plastici deboli, metalli molto durosi, fibre di vetro o altri materiali fragili. Si consiglia l'utilizzo di materiali robusti e resistenti che possono garantire la sicurezza e la durabilità del cestello della macchina.


In [16]:
rag_chain_compressor = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)


In [17]:
print(rag_chain_compressor.invoke("Quali materiali è proibito utilizzare nel cestello della macchina?"))  # What materials are prohibited from being used in the machine's basket?


 Nel cestello della macchina non sono consentiti materiali infusibili, quali ad esempio legno o plastiche sintetiche. Il materiale di solito utilizzato è l'acciaio inossidabile avente una certa resistenza calorifica, spesso indicato come "cestello". Altri materiali possibili sono i metalli leggeri quali alluminio o titanio. È consigliabile sempre consultare le informazioni fornite dall'industria del cestello o dal costruttore della macchina per ulteriori dettagli sulla scelta di materiale corretto.


In [18]:
queries = [
    "Per quale scopo è stata progettata la macchina del del GIRACASTAGNE AUTOMATICO ROTOMARR?",  # What purpose was the machine designed for?
    "Quali sono i passaggi da seguire prima di procedere alla regolazione con GIRACASTAGNE AUTOMATICO ROTOMARR?", # What are the steps to take before making adjustment?
    "A quale altezza dovrebbe essere regolato il cestello (C) rispetto al piano di cottura?"  #At what height should the basket (C) be adjusted relative to the cooking surface?
    #"Quali sono le caratteristiche del GIRACASTAGNE AUTOMATICO ROTOMARR?"  # What are characteristic of GIRACASTAGNE AUTOMATICO ROTOMARR?
    #"Menziona i materiali il cui utilizzo è proibito nel cestello della macchina."  # Mention materials that are prohibited from being used in the machine's basket
]


In [19]:
#Simple Retriver Chain
responses = rag_chain.map().invoke(queries)

for response in responses:
    print("====================RAG RESPONSE===============================\n")
    print(response)
    print("===========================================================\n\n")


====================RAG RESPONSE===============================

 La macchina del taglio del castagno Giracastagne Automatico Rotomarr è stata progettata per facilitare il processo di raccolta e taglio dei castagni, offrendo una soluzione automatica e efficiente a tale scopo. In pratica, consente alle persone di raccogliere i castagni in modo meno impegnativo e più velocemente rispetto al lavoro manuale.


====================RAG RESPONSE===============================

1. Verifica che la macchina sia disconnessa dalla fonte di energia elettrica per garantire la sicurezza durante l'operazione.
2. Disassembla le parti esterne del giracastagne automatico Rotomarr per accedere alle parti interne necessarie alla regolazione.
3. Segui le istruzioni fornite dal manuale di utilizzo o da un tecnico qualificato in merceologia per capire cosa devi regolare, come la distanza tra i denti della cerniera e il corpo del giracastagne automatico.
4. Regola le parti interne del giracastagne automatico s

In [20]:
# Compressor Retriever Chain
responses = rag_chain_compressor.map().invoke(queries)
for response in responses:
    print("====================COMPRESSOR RAG RESPONSE=============================================================================\n\n")
    print(response)
    print("=======================================================================================================================\n\n")


====================COMPRESSOR RAG RESPONSE=============================================================================


 La macchina Giracastagne Automatico Rotomarr è stata progettata per il raccolto automatizzato delle castagne.


====================COMPRESSOR RAG RESPONSE=============================================================================


 Per procedere alla regolazione del Giracastagne Automatico Rotomarr, segui i seguenti passaggi:

1. Disconnetti l'apparecchio dall'origine di alimentazione elettrica prima di qualsiasi lavoro di manutenzione o regolazione.
2. Rimuovi eventuali oggetti o detriti dalla griglia di alimentazione della macchina per assicurarti che sia pulita e sicura da utilizzare.
3. Sposta la macchina in una posizione sicura dove puoi lavorare su di essa senza rischi di ferire qualcuno o danneggiare l'apparecchio.
4. Verifica che tutti gli elementi meccanici della macchina siano funzionanti correttamente. In particolare, controlla se le ruote del girad

# Glider Evaluation Process, LLM-as-Judge

In [24]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
import json

class GLIDEREvaluator:
    def __init__(self):
        self.model_name = "PatronusAI/glider"
        self.pipe = pipeline(
            "text-generation",
            model=self.model_name,
            max_new_tokens=2048,
            device=-1,  # Use CPU instead of cuda
            return_full_text=False
        )

        self.evaluation_prompt = """Analyze the following pass criteria carefully and score the text based on the rubric defined below.

        To perform this evaluation, you must:
        1. Understand the text tags, pass criteria and rubric thoroughly.
        2. Review the finer details of the text and the rubric.
        3. Compare the tags to be evaluated to the score descriptions in the rubric.
        4. Pay close attention to small details that might impact the final score.
        5. Write a detailed reasoning justifying your evaluation in a bullet point format.
        6. The reasoning must summarize the overall strengths and weaknesses while quoting exact phrases.
        7. Output a list of words or phrases that you believe are the most important in determining the score.
        8. Assign a final score based on the scoring rubric.

        Data to evaluate:
        <CONTEXT>
        {context}
        </CONTEXT>

        <USER INPUT>
        {question}
        </USER INPUT>

        <MODEL OUTPUT>
        {answer}
        </MODEL OUTPUT>

        Pass Criteria:
        - Relevance: The answer should be directly relevant to the question
        - Faithfulness: The answer should be supported by the context
        - Completeness: The answer should cover all aspects of the question
        - Coherence: The answer should be well-structured and logical
        - Citation: The answer should reference specific parts of the context when needed

        Rubric:
        5: Exceptional - Perfect relevance, completely faithful, comprehensive coverage
        4: Strong - High relevance, mostly faithful, good coverage
        3: Adequate - Moderate relevance, somewhat faithful, partial coverage
        2: Poor - Low relevance, multiple faithfulness issues, incomplete
        1: Unacceptable - Irrelevant, unfaithful, or severely incomplete

        Your output must in the following format:
        <reasoning>
        [Detailed reasoning justifying your evaluation in a bullet point format]
        </reasoning>
        <highlight>
        [List of key phrases that determined the score]
        </highlight>
        <score>
        [Final integer score 1-5]
        </score>
        """

    def evaluate_response(self, context, question, answer):
        """
        Evaluate a single RAG response using GLIDER
        """
        prompt = self.evaluation_prompt.format(
            context=context,
            question=question,
            answer=answer
        )

        messages = [{"role": "user", "content": prompt}]
        result = self.pipe(messages)[0]['generated_text']

        # Parse the evaluation results
        reasoning = self._extract_section(result, "reasoning")
        highlights = self._extract_section(result, "highlight")
        score = self._extract_section(result, "score")

        return {
            "reasoning": reasoning,
            "highlights": highlights,
            "score": int(score) if score.strip().isdigit() else None
        }

    def _extract_section(self, text, section_name):
        """Extract content between section tags"""
        start_tag = f"<{section_name}>"
        end_tag = f"</{section_name}>"
        try:
            start = text.index(start_tag) + len(start_tag)
            end = text.index(end_tag)
            return text[start:end].strip()
        except ValueError:
            return ""


In [25]:
def evaluate_rag_pipeline(rag_chain_compressor, test_questions, contexts=None):
    """
    Evaluate the RAG pipeline using GLIDER

    Args:
        pipeline: The RAG pipeline instance
        test_questions: List of test questions
        contexts: Optional list of contexts for each question

    Returns:
        dict: Evaluation results including scores and analysis
    """
    evaluator = GLIDEREvaluator()
    results = []

    for i, question in enumerate(test_questions):
        # Get RAG response
        retrieved_docs = compression_retriever.invoke(question)
        response = rag_chain_compressor.invoke(question)
        print(f" RAG Response:{response}")

        # Get context used
        context = "\n".join([doc.page_content for doc in retrieved_docs])
        #print(f"Context:{context}")
        # Evaluate using GLIDER
        evaluation = evaluator.evaluate_response(
            context=context,
            question=question,
            answer=response
        )

        results.append({
            "question": question,
            "response": response,
            "evaluation": evaluation
        })

    # Calculate aggregate metrics
    scores = [r["evaluation"]["score"] for r in results if r["evaluation"]["score"]]
    aggregate_metrics = {
        "average_score": sum(scores) / len(scores) if scores else 0,
        "max_score": max(scores) if scores else 0,
        "min_score": min(scores) if scores else 0,
        "total_evaluated": len(scores)
    }

    return {
        "detailed_results": results,
        "aggregate_metrics": aggregate_metrics
    }


# Ollama Model RAG Response Evaluation

In [26]:
test_questions = [
    "Per quale scopo è stata progettata la macchina del del GIRACASTAGNE AUTOMATICO ROTOMARR?",  # What purpose was the machine designed for?
    "Quali sono i passaggi da seguire prima di procedere alla regolazione con GIRACASTAGNE AUTOMATICO ROTOMARR?", # What are the steps to take before making adjustment?
    "A quale altezza dovrebbe essere regolato il cestello (C) rispetto al piano di cottura?"  #At what height should the basket (C) be adjusted relative to the cooking surface?
    #"Quali sono le caratteristiche del GIRACASTAGNE AUTOMATICO ROTOMARR?"  # What are characteristic of GIRACASTAGNE AUTOMATICO ROTOMARR?
    #"Menziona i materiali il cui utilizzo è proibito nel cestello della macchina."  # Mention materials that are prohibited from being used in the machine's basket
]

ollama_llm = OllamaLLM(model='mistral:7b')

# Simple retriever
retriver = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":5})

compressor = LLMChainFilter.from_llm(llm=ollama_llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriver)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriver)

rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
    )

evaluation_results = evaluate_rag_pipeline(rag_chain, test_questions)

# Print results
print("\nEvaluation Results:")
print("\nAggregate Metrics:")
print(json.dumps(evaluation_results["aggregate_metrics"], indent=2))

print("\nDetailed Results:")
for result in evaluation_results["detailed_results"]:
    print(f"\nQuestion: {result['question']}")
    print(f"Score: {result['evaluation']['score']}")
    print("Reasoning:")
    print(result['evaluation']['reasoning'])
    print("Key Highlights:")
    print(result['evaluation']['highlights'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


 RAG Response: La Giracastagne Automatico Rotomarr è stata progettata per il compito di raccolta automatica delle castagne. Essa ha lo scopo di raccogliere le castagne cadute a terra, trasformandole in una produzione efficace e non impegnativa per l'operatore.


KeyboardInterrupt: 

# Groq Mixtral Model RAG Response Evaluation

In [ ]:
groq_llm = ChatGroq(model="mixtral-8x7b-32768",
                    temperature=0,
                    max_tokens=None,
                    timeout=None,
                    max_retries=5,)

# Simple retriever
retriver = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":5})

compressor = LLMChainFilter.from_llm(llm=groq_llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriver)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriver)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | groq_llm
    | StrOutputParser()
    )

evaluation_results = evaluate_rag_pipeline(rag_chain, test_questions)

# Print results
print("\nEvaluation Results:")
print("\nAggregate Metrics:")
print(json.dumps(evaluation_results["aggregate_metrics"], indent=2))

print("\nDetailed Results:")
for result in evaluation_results["detailed_results"]:
    print(f"\nQuestion: {result['question']}")
    print(f"Score: {result['evaluation']['score']}")
    print("Response:")
    print(result['response'])
    print("Reasoning:")
    print(result['evaluation']['reasoning'])
    print("Key Highlights:")
    print(result['evaluation']['highlights'])
    print("=======================================================================================================================================")
    print("\n\n")

# Delete Folder

In [ ]:
import shutil
import os

def delete_folder(folder_path):
    """
    Deletes a folder and its contents, even if it's not empty.

    Args:
        folder_path (str): The path to the folder you want to delete.
    """
    try:
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting folder: {e}")

# Example usage:
folder_to_delete = "/content/chromadb"  # Replace with the actual path
delete_folder(folder_to_delete)